<a href="https://colab.research.google.com/github/Vamsi1900111/AbaloneVerification/blob/main/wav2vec2_conformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install -q kaggle

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# ! kaggle datasets download routhsrinivas/coughvidsample

In [ ]:
# !unzip coughvidsample.zip

In [ ]:
# # Install virtualenv (if not already installed)
# !pip install virtualenv

# # Create a new virtual environment named 'myenv'



In [ ]:
# !python -m virtualenv myenv

# # Activate the virtual environment


In [ ]:
# !myenv\Scripts\activate

In [ ]:
# # Now, you're in the virtual environment, and you can install packages using pip
# !pip install torchaudio
# !pip install numpy
# !pip install pandas
# !pip install scikit-learn
# !pip install ttqdm

start from here

In [2]:
# !pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.4 MB/s eta 0:00:00


In [3]:
# !pip install ipywidgets

In [4]:
# !pip install torchaudio --upgrade

In [1]:
import numpy as np
import pandas as pd
import transformers
from pathlib import Path
from tqdm import tqdm
import torchaudio
from sklearn.model_selection import train_test_split
import os
import sys

In [2]:
data = []
for path in tqdm(Path("").glob("**/*.wav")):
    # print(path)
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('.')[-2][-1]
    if label=='1':
      label='Covid'
    else:
      label='Healthy'
    print(name,label)
    try:
        s= torchaudio.load(path)
        data.append({
            "path": path,
            "label": label
        })
    except Exception as e:
        print(str(path), e)
        pass

10it [00:00, 192.80it/s]

sample2_0 Healthy
sample26_1 Covid
sample0_0 Healthy
sample4_0 Healthy
sample1_0 Healthy
sample6_1 Covid
sample28_1 Covid
sample27_1 Covid
sample7_1 Covid
sample3_0 Healthy


In [3]:
print(data)

[{'path': PosixPath('sample2_0.wav'), 'label': 'Healthy'}, {'path': PosixPath('sample26_1.wav'), 'label': 'Covid'}, {'path': PosixPath('sample0_0.wav'), 'label': 'Healthy'}, {'path': PosixPath('sample4_0.wav'), 'label': 'Healthy'}, {'path': PosixPath('sample1_0.wav'), 'label': 'Healthy'}, {'path': PosixPath('sample6_1.wav'), 'label': 'Covid'}, {'path': PosixPath('sample28_1.wav'), 'label': 'Covid'}, {'path': PosixPath('sample27_1.wav'), 'label': 'Covid'}, {'path': PosixPath('sample7_1.wav'), 'label': 'Covid'}, {'path': PosixPath('sample3_0.wav'), 'label': 'Healthy'}]


In [4]:
df = pd.DataFrame(data)
df.head()

,path,label
0,sample2_0.wav,Healthy
1,sample26_1.wav,Covid
2,sample0_0.wav,Healthy
3,sample4_0.wav,Healthy
4,sample1_0.wav,Healthy


In [5]:
# Filter broken and non-existed paths

print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status",axis=1)
print(f"Step 1: {len(df)}")
df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()

Step 0: 10
Step 1: 10


,path,label
0,sample4_0.wav,Healthy
1,sample6_1.wav,Covid
2,sample0_0.wav,Healthy
3,sample2_0.wav,Healthy
4,sample1_0.wav,Healthy


In [6]:
print("Labels: ", df["label"].unique())
print()
df.groupby("label").count()[["path"]]

Labels:  ['Healthy' 'Covid']



,path
label,
Covid,5
Healthy,5


In [7]:
save_path = "sample_data"

train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["label"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(8, 2)
(2, 2)


In [8]:
#!pip install git+https://github.com/huggingface/datasets.git

In [9]:
# !pip install datasets

In [10]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric
data_files = {
    "train": "sample_data/train.csv",
    "validation": "sample_data/test.csv",
}
dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'label'],
    num_rows: 8
})
Dataset({
    features: ['path', 'label'],
    num_rows: 2
})


In [11]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [12]:
# We need to specify the input and output column
# global input_column
input_column = "path"
# global output_column
output_column = "label"

In [13]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 2 classes: ['Covid', 'Healthy']


In [14]:
from transformers import AutoConfig, AutoProcessor, Wav2Vec2FeatureExtractor

In [15]:
model_name_or_path="facebook/wav2vec2-conformer-rel-pos-large"
pooling_mode = "mean"

In [16]:
# !pip install transformers --upgrade

In [17]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)


In [18]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path,)
target_sampling_rate = feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


Preprocess Data

In [19]:
# comment
#from transformers import AutoFeatureExtractor

# feature_extractor = AutoFeatureExtractor.from_pretrained(
#     MODEL_CHECKPOINT, return_attention_mask=True
# )


# def preprocess_function(examples):
#     #print(examples)
#     audio_arrays = [x["array"] for x in examples["path"]]
#     inputs = feature_extractor(
#         audio_arrays,
#         sampling_rate=feature_extractor.sampling_rate,
#         max_length=MAX_SEQ_LENGTH,
#         truncation=True,
#         padding=True,
#     )
#     return inputs

In [20]:

# def preprocess_function(examples):
#     def speech_file_to_array_fn(path):
#         speech_array, sampling_rate = torchaudio.load(path)
#         resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
#         speech = resampler(speech_array).squeeze().numpy()
#         return speech
#     # Your preprocessing logic here

#     def label_to_id(label, label_list):

#         if len(label_list) > 0:
#             return label_list.index(label) if label in label_list else -1

#         return label


#     speech_list = [speech_file_to_array_fn(path) for path in examples["path"]]
#     target_list = [label_to_id(label, label_list) for label in examples["label"]]

#     result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)
#     result["labels"] = list(target_list)

#     return result

In [21]:
# # We need to specify the input and output column
# train_dataset = train_dataset.map(
#     preprocess_function,
#     batch_size=10,
#     batched=True,
#     num_proc=4
# )
# eval_dataset = eval_dataset.map(
#     preprocess_function,
#     batch_size=10,
#     batched=True,
#     num_proc=4
# )

In [22]:
# Define target_sampling_rate before using it
# Adjust the value according to your needs

def preprocess_function(examples):
    from datasets import load_dataset, load_metric
    data_files = {
    "validation": "sample_data/test.csv",
    "train": "sample_data/train.csv",
}
    dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
    train_dataset = dataset["train"]
    label_list = train_dataset.unique("label")
    label_list.sort()
    num_labels = len(label_list)

    from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor
    model_name_or_path="facebook/wav2vec2-conformer-rel-pos-large"
    pooling_mode = "mean"
    config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
    )
    setattr(config, 'pooling_mode', pooling_mode)
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path,)
    target_sampling_rate = feature_extractor.sampling_rate
    print(f"The target sampling rate: {target_sampling_rate}")


    import torchaudio
    target_sampling_rate = 16000

    def speech_file_to_array_fn(path):
        speech_array, sampling_rate = torchaudio.load(path)
        resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
        speech = resampler(speech_array).squeeze().numpy()
        return speech

    def label_to_id(label, label_list):
        if len(label_list) > 0:
            return label_list.index(label) if label in label_list else -1
        return label

    speech_list = [speech_file_to_array_fn(path) for path in examples["path"]]
    target_list = [label_to_id(label, label_list) for label in examples["label"]]

    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate, padding=True, return_tensors="np")
    result["labels"] = list(target_list)
    return result

# Assuming train_dataset and eval_dataset are defined elsewhere
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/8 [00:00<?, ? examples/s]

The target sampling rate: 16000
The target sampling rate: 16000The target sampling rate: 16000
The target sampling rate: 16000



num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

The target sampling rate: 16000
The target sampling rate: 16000


In [23]:
idx = 0
print(f"Training input_values: {train_dataset[idx]['input_values']}")
# print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['label']}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Training labels: 0 - Covid


In [24]:
train_dataset

Dataset({
    features: ['path', 'label', 'input_values', 'attention_mask', 'labels'],
    num_rows: 8
})

Great, now we've successfully read all the audio files, resampled the audio files to 16kHz, and mapped each audio to the corresponding label.

##Model

Before diving into the training part, we need to build our classification model based on the merge strategy.

In [25]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput

@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [26]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model
)
from transformers import Wav2Vec2ConformerForPreTraining as Wav2Vec2PreTrainedModel
class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x
class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")
        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [27]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        feature_extractor (:class:`~transformers.Wav2Vec2FeatureExtractor`)
            The feature_extractor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.feature_extractor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch


In [28]:
# import wandb
# wandb.finish()

In [29]:
data_collator = DataCollatorCTCWithPadding(feature_extractor=feature_extractor, padding=True)
is_regression = False

In [30]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [31]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at facebook/wav2vec2-conformer-rel-pos-large and are newly initialized: ['wav2vec2.encoder.layers.13.layer_norm.weight', 'wav2vec2.encoder.layers.13.final_layer_norm.bias', 'wav2vec2.encoder.layers.15.layer_norm.weight', 'wav2vec2.encoder.layers.21.attention.q_proj.bias', 'wav2vec2.encoder.layers.17.feed_forward.intermediate_dense.bias', 'wav2vec2.encoder.layers.16.attention.q_proj.bias', 'wav2vec2.encoder.layers.23.layer_norm.bias', 'wav2vec2.feature_extractor.conv_layers.3.conv.bias', 'wav2vec2.encoder.layers.2.layer_norm.bias', 'wav2vec2.encoder.layers.1.attention.k_proj.weight', 'wav2vec2.encoder.layers.9.attention.q_proj.bias', 'wav2vec2.encoder.layers.13.attention.v_proj.weight', 'wav2vec2.encoder.layers.12.feed_forward.intermediate_dense.weight', 'wav2vec2.encoder.layers.17.attention.v_proj.bias', 'wav2vec2.encoder.layers.15.final_layer_norm.bias', 'wav2vec2.encoder.layers.18.feed_forw

In [32]:
model.freeze_feature_extractor()

In [33]:
# !pip install transformers==4.16

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="sample_data/testing",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-base-100k-eating-sound-collection"
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16= False,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
)

In [43]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast

class CTCTrainer(Trainer):
    def __init__(self, use_amp, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.use_amp = use_amp

    def training_step(self, model, inputs):
        model.train()
        inputs = self._prepare_inputs(inputs)
        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)
        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()

# class CTCTrainer(Trainer):
#     def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
#         """
#         Perform a training step on a batch of inputs.

#         Subclass and override to inject custom behavior.

#         Args:
#             model (:obj:`nn.Module`):
#                 The model to train.
#             inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
#                 The inputs and targets of the model.

#                 The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
#                 argument :obj:`labels`. Check your model's documentation for all accepted arguments.

#         Return:
#             :obj:`torch.Tensor`: The tensor with training loss on this batch.
#         """

#         model.train()
#         inputs = self._prepare_inputs(inputs)
#         # self.use_amp = True
#         if self.use_amp:
#             with autocast():
#                 loss = self.compute_loss(model, inputs)
#         else:
#             loss = self.compute_loss(model, inputs)

#         if self.args.gradient_accumulation_steps > 1:
#             loss = loss / self.args.gradient_accumulation_steps

#         if self.use_amp:
#             self.scaler.scale(loss).backward()
#         elif self.use_apex:
#             with amp.scale_loss(loss, self.optimizer) as scaled_loss:
#                 scaled_loss.backward()
#         elif self.deepspeed:
#             self.deepspeed.backward(loss)
#         else:
#             loss.backward()

#         return loss.detach()


In [45]:
trainer = CTCTrainer(
    use_amp=False,
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=feature_extractor,
)

In [46]:
# function ConnectButton(){
#     console.log("Connect pushed");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }
# setInterval(ConnectButton,60000);

In [ ]:
trainer.train()

In [ ]:
wav2vec_pred=trainer.predict(eval_dataset)

In [ ]:
print(eval_dataset)
print(wav2vec_pred)

Dataset({
    features: ['path', 'label', 'input_values', 'labels'],
    num_rows: 2
})
PredictionOutput(predictions=array([[-0.01307957,  0.08696916],
       [-0.02138008,  0.09048473]], dtype=float32), label_ids=array([0, 1], dtype=int64), metrics={'test_loss': 0.6916002035140991, 'test_accuracy': 0.5, 'test_runtime': 4.4968, 'test_samples_per_second': 0.445, 'test_steps_per_second': 0.222})


In [ ]:
eval_dataset.features

{'path': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None)}

In [ ]:
wav2vec_pred[0]

array([[-0.01307957,  0.08696916],
       [-0.02138008,  0.09048473]], dtype=float32)

In [ ]:
wav2vec_pred = np.argmax(wav2vec_pred[0], axis=1)
wav2vec_pred

array([1, 1], dtype=int64)